Run this notebook to

- xx



In [ ]:
# accuracy above >= 62% in non-strategic task: 7, 19, 1, 11, 20, 17, 18, 9, 6, 2

In [ ]:
# accuracy above >= 62% in strategic task: 1, 2, 19, 18, 11, 3, 17

In [ ]:
# accuracy above >= 64% in non-strategic task: 7, 19, 1, 11, 20, 17

In [ ]:
# accuracy above >= 64% in strategic task: 1, 2, 19, 18, 11, 3

## Import stuff and set parameteres

In [1]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import seaborn as sns
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr 
from itertools import chain, zip_longest

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.model_selection import KFold, cross_val_predict, train_test_split 
from sklearn.metrics import mean_squared_error, r2_score

import mne
from mne.stats import permutation_t_test
mne.set_log_level('warning') 

#%matplotlib qt
%matplotlib inline

input_dir = 'TaskstimulusEpochsMastoids'


In [3]:
def load_subj_eeg(path, file, downsample=None):
    fp = os.path.join(path, '%s-epo.fif' % file)
    print('>>> Loading %s' % fp)
    epochs = mne.read_epochs(fp, preload=True)
    if downsample is not None:
        epochs = epochs.resample(downsample)
    return epochs

def load_all_eeg(path, files, downsample=None):
    subject_epochs = [load_subj_eeg(path, file, downsample=downsample) for file in files]
    epochs = mne.epochs.concatenate_epochs(subject_epochs)
    return epochs

In [4]:
participant_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21] # 14 was excluded due to noise

sessions = [1, 2]

partners = ['"overconfident"', '"underconfident"']

epoch_start = 0.25
epoch_end = 0.75

## Correlation reaction time and confidence

In [5]:
%%capture

strategic_correlations = []
non_strategic_correlations = []

for session in sessions:
#     # accuracy >=62%
#     if session == 1:
#         participant_numbers = [1, 19, 11, 3, 17, 20, 18, 6, 2]
#     if session == 2:
#         participant_numbers = [2, 18, 7, 19, 1, 11, 17, 9]
        
    # accuracy >=64%
    if session == 1:
        participant_numbers = [1, 19, 11, 3, 20]
    if session == 2:
        participant_numbers = [2, 18, 7, 19, 1, 11, 17]
        
    
    for subject in participant_numbers: 
        participant_files = []
        participant_files.append('%i_%i' % (subject, session))
        epochs = load_subj_eeg(path='%s/' % input_dir, file=participant_files[0])

        y = epochs.metadata['participant_confidence'].to_numpy()
        x = epochs.metadata['decision_rt'].to_numpy()

        a = pearsonr(x, y)[0]
        print(a)

        condition = epochs.metadata['condition'].unique()
        if condition == "s":
            strategic_correlations.append(a)
        if condition == "ns":
            non_strategic_correlations.append(a)


In [6]:

print('\nStrategic average correlation confidence and reaction time: %f' % (sum(strategic_correlations) / len(strategic_correlations),))


print('\nNon-strategic average correlation confidence and reaction time: %f' % (sum(non_strategic_correlations) / len(non_strategic_correlations),))



Strategic average correlation confidence and reaction time: -0.338381

Non-strategic average correlation confidence and reaction time: -0.250031


## Linear regression predicting confidence

In [7]:
%%capture

all_lr_scores = []

for session in sessions:
#     # accuracy >=62%
#     if session == 1:
#         participant_numbers = [1, 19, 11, 3, 17, 20, 18, 6, 2]
#     if session == 2:
#         participant_numbers = [2, 18, 7, 19, 1, 11, 17, 9]
        
    # accuracy >=64%
    if session == 1:
        participant_numbers = [1, 19, 11, 3, 20]
    if session == 2:
        participant_numbers = [2, 18, 7, 19, 1, 11, 17]
        

    for subject in participant_numbers:   
    #     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    #     print('\033[0m')

        # merge stimulus epochs from both sessions for the participant - TREATING EVERYONE INDIVIDUALLY
        participant_files = []
        participant_files.append('%i_%i' % (subject, session))
        epochs = load_subj_eeg(path='%s/' % input_dir, file=participant_files[0])

        # crop epochs
        epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)

        # independent variables
        # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
        X = epochs.get_data()

        n_epochs = X.shape[0]
        n_channels = X.shape[1]
        n_timepoints = X.shape[2]

        # collapse across trials and timepoints
        X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)
        X = X.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

        # dependent variable as numpy array (n_epochs,)
        y = epochs.metadata['participant_confidence'].to_numpy()
        # extend from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
        y = np.repeat(y, n_timepoints)

        # fit regression
        reg = LinearRegression().fit(X, y)
        score = reg.score(X, y)
    #     print(score)

    #     # regression betas
    #     betas = reg.coef_
    #     #print(betas)

    #     mne.viz.plot_topomap(data=betas, pos=epochs.info)

        all_lr_scores.append(score)


In [8]:

print('Average R-squared linear regression: %f' % (sum(all_lr_scores) / len(all_lr_scores),))


Average R-squared linear regression: 0.140731


## Cross-validation random split across timepoints

In [ ]:
all_cv_scores = []

for session in sessions:
#     # accuracy >=62%
#     if session == 1:
#         participant_numbers = [1, 19, 11, 3, 17, 20, 18, 6, 2]
#     if session == 2:
#         participant_numbers = [2, 18, 7, 19, 1, 11, 17, 9]
        
    # accuracy >=64%
    if session == 1:
        participant_numbers = [1, 19, 11, 3, 20]
    if session == 2:
        participant_numbers = [2, 18, 7, 19, 1, 11, 17]

    for subject in participant_numbers:   
    #     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    #     print('\033[0m')

        # merge stimulus epochs from both sessions for the participant - TREATING EVERYONE INDIVIDUALLY
        participant_files = []
        participant_files.append('%i_%i' % (subject, session))
        epochs = load_subj_eeg(path='%s/' % input_dir, file=participant_files[0])
    
        # crop epochs
        epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)



        ###### INDEPENDENT VARIABLES
        # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times) (600, 64, 127) (250 Hz --> every 4 ms)
        X = epochs.get_data()
        n_channels = X.shape[1]
        n_timepoints = X.shape[2]
        X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)



        ###### DEPENDENT VARIABLES
        ## this version is splitting data randomly by timepoints - topoplots etc look v similar
        # 50% training data, 50% test data
        X = X.reshape(-1, n_channels)
        y = epochs.metadata['participant_confidence'].to_numpy()
        y = np.repeat(y, n_timepoints)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)



        ###### TRAIN CLASSIFIER
        regressor = LinearRegression()  
        regressor.fit(X_train, y_train) #training the algorithm

        # regression betas
        betas = regressor.coef_

        # predict y from X_test
        y_pred = regressor.predict(X_test)    


        ###### PLOT
        # plot topoplot with beta weights
        mne.viz.plot_topomap(data=betas, pos=epochs.info)
        print("Beta weights for each channel \n\n")

        # plot topoplot with sensor projections
        # get out sensor projections (see Parra et al paper)
        y = y_pred
        x = X_test
        a =  y.T@x * ( 1 / (y.T@y) )
        mne.viz.plot_topomap(data=a, pos=epochs.info)
        print("Sensor projections \n\n")

        # plot confidence signal ERP over time (use betas rather than a weights)
        across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
        weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
        average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
        x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
        plt.plot(x_axis, average_weighted_erp)
        plt.show();
        print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")

    #     # plot normal ERP over time
    #     across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
    #     average_across_epochs_erp = across_epochs_erp.mean(axis=0) # compute the average across electrodes 
    #     x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
    #     plt.plot(x_axis, average_across_epochs_erp)
    #     plt.show()
    #     print("Average ERP (not weighted) \n\n")


        print('R-squared:', r2_score(y_test, y_pred))  
        all_cv_scores.append(r2_score(y_test, y_pred))



In [ ]:

print('Average R-squared cross validation random timepoint split: %f' % (sum(all_cv_scores) / len(all_cv_scores),))


## Cross-validation training on odd (even) trials, testing on even (odd) trials

In [ ]:
all_scores_test = []
all_scores_test_trialwise = []
all_scores_train = []
all_scores_train_trialwise = []


for session in sessions:
#     # accuracy >=62%
#     if session == 1:
#         participant_numbers = [1, 19, 11, 3, 17, 20, 18, 6, 2]
#     if session == 2:
#         participant_numbers = [2, 18, 7, 19, 1, 11, 17, 9]
        
    # accuracy >=64%
    if session == 1:
        participant_numbers = [1, 19, 11, 3, 20]
    if session == 2:
        participant_numbers = [2, 18, 7, 19, 1, 11, 17]

    for subject in participant_numbers:   
    #     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    #     print('\033[0m')

        # merge stimulus epochs from both sessions for the participant - TREATING EVERYONE INDIVIDUALLY
        participant_files = []
        participant_files.append('%i_%i' % (subject, session))
        epochs = load_subj_eeg(path='%s/' % input_dir, file=participant_files[0])
    
        # crop epochs
        epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)


        ######################################################################
        ####################### INDEPENDENT VARIABLE ######################### 
        ######################################################################

        # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times) (600, 64, 127) (250 Hz --> every 4 ms)
        X = epochs.get_data()
        n_epochs = X.shape[0]
        n_channels = X.shape[1]
        n_timepoints = X.shape[2]
        X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels) (600, 127, 64)

        # divide into odd and even trials (3D array) (300, 127, 64)
        X_odd_trials = X[1::2, :, :] # select every second epoch (start:stop:step, :, :)
        X_even_trials = X[0::2, :, :] # select every other second epoch

        # collapse across trials and timepoints (2D array) (triple checked this and it is correctly sorted as e1t1, e1t2, e1t3, ...)
        X_odd_per_timepoint = X_odd_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features
        X_even_per_timepoint = X_even_trials.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features


        ######################################################################
        ######################## DEPENDENT VARIABLE ########################## 
        ######################################################################

        # dependent variable as numpy array (n_epochs,)
        y = epochs.metadata['participant_confidence'].to_numpy()

        # divide into odd and even trials
        y_odd_trials = y[1::2]
        y_even_trials = y[0::2]

        # make from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
        y_odd_per_timepoint = np.repeat(y_odd_trials, n_timepoints)
        y_even_per_timepoint = np.repeat(y_even_trials, n_timepoints)


        ######################################################################
        ####################### TRAINING ON ODD TRIALS ####################### 
        ######################################################################

        print('\033[1m' + "\n\n ______ train on odd trials ______\n")
        print('\033[0m')

        X_train_trials = X_odd_trials
        X_train_per_timepoint = X_odd_per_timepoint

        X_test_trials = X_even_trials
        X_test_per_timepoint = X_even_per_timepoint


        y_train_trials = y_odd_trials
        y_train_per_timepoint = y_odd_per_timepoint

        y_test_trials = y_even_trials
        y_test_per_timepoint = y_even_per_timepoint



        # train classifier
        regressor = LinearRegression()  
        regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
        betas = regressor.coef_   
        intercept = regressor.intercept_

        # predict y from X_test 
        # OPTION 1
        # use predict function on individual timepoints
        y_pred_per_timepoint = regressor.predict(X_test_per_timepoint) 
        # then average predicted values within an epoch
        epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)

        # OPTION 2
        # use betas and intercept on individual timepoints   
        y_pred_per_timepoint_2 = np.sum(X_test_per_timepoint * betas, axis=1) + intercept 
        # then average predicted values within an epoch
        epoch_y_pred_2 = np.mean(y_pred_per_timepoint_2.reshape(-1, n_timepoints), axis=1)

        # OPTION 3
        # use predict function on epoch average
        epoch_y_pred_3 = regressor.predict(np.mean(X_test_trials, axis=1)) # take mean voltage along n_times axis (n_epochs, n_times, n_channels)

        # OPTION 4
        # use betas and intercept on epoch average
        epoch_y_pred_4 = np.sum(np.mean(X_test_trials, axis=1) * betas, axis=1) + intercept 

    #     # sanity check - all options give the same predicted y per epoch
    #     print(epoch_y_pred_1[0:6])
    #     print(epoch_y_pred_2[0:6])
    #     print(epoch_y_pred_3[0:6])
    #     print(epoch_y_pred_4[0:6])


        print('\033[1m' + "\n test data")
        print('\033[0m')
        # R squared for timepoints
        print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    #     # sanity check - manual computation gives same R-squared
    #     actual_minus_predicted = sum((y_test_per_timepoint - y_pred_per_timepoint_1)**2)
    #     actual_minus_actual_mean = sum((y_test_per_timepoint - y_test_per_timepoint.mean())**2)
    #     r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
    #     print('R-squared timepoint data:', r2)

        # R squared for trials
        print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
    #     # sanity check - manual computation gives same R-squared
    #     actual_minus_predicted = sum((y_test_trials - epoch_y_pred_1)**2)
    #     actual_minus_actual_mean = sum((y_test_trials - y_test_trials.mean())**2)
    #     r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
    #     print('R-squared trial data:', r2)
        # plot y against predicted y
        sns.regplot(x=y_test_trials, y=epoch_y_pred, scatter_kws={'alpha':0.5})
        plt.xlabel('Reported confidence')
        plt.ylabel('Classifier prediction')
        plt.title('Reported trial confidence against predicted trial confidence test data')
        plt.show();


        # R squared for training data
        print('\033[1m' + "\n train data")
        print('\033[0m')
        # R squared for timepoints
        y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
        print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
        # R squared for trials
        epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
        print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
        # plot y against predicted y
        sns.regplot(x=y_train_trials, y=epoch_y_pred_train, scatter_kws={'alpha':0.5})
        plt.xlabel('Reported confidence')
        plt.ylabel('Classifier prediction')
        plt.title('Reported trial confidence against predicted trial confidence train data')
        plt.show();

        all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
        all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
        all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
        all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))


        ####################### PLOTTING ####################### 

        # plot topoplot with beta weights
        mne.viz.plot_topomap(data=betas, pos=epochs.info)
        print("Beta weights for each channel \n\n")


        # plot topoplot with sensor projections (see Parra et al paper for equation)
        y = y_pred_per_timepoint
        x = X_test_per_timepoint
        a =  y.T@x * ( 1 / (y.T@y) )
        mne.viz.plot_topomap(data=a, pos=epochs.info)
        print("Sensor projections \n\n")

    #     # sanity check - plotting with trial data rather than timepoint data gives the same plot
    #     y = epoch_y_pred
    #     x = np.mean(X_test_trials, axis=1) 
    #     a =  y.T@x * ( 1 / (y.T@y) )
    #     mne.viz.plot_topomap(data=a, pos=epochs.info)
    #     print("Sensor projections trial data \n\n")

        # plot confidence signal ERP over time (use betas rather than a weights)
        across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
        weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
        average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
        x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
        plt.plot(x_axis, average_weighted_erp)
        plt.show()
        print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")



        ######################################################################
        ####################### TRAINING ON EVEN TRIALS ###################### 
        ######################################################################

        print('\033[1m' + "\n\n ______ train on even trials ______\n")
        print('\033[0m')

        X_train_trials = X_even_trials
        X_train_per_timepoint = X_even_per_timepoint

        X_test_trials = X_odd_trials
        X_test_per_timepoint = X_odd_per_timepoint


        y_train_trials = y_even_trials
        y_train_per_timepoint = y_even_per_timepoint

        y_test_trials = y_odd_trials
        y_test_per_timepoint = y_odd_per_timepoint



        # train classifier
        regressor = LinearRegression()  
        regressor.fit(X_train_per_timepoint, y_train_per_timepoint) #training the algorithm on every timepoint
        betas = regressor.coef_
        intercept = regressor.intercept_

        # predict y from X_test 
        # use predict function on individual timepoints
        y_pred_per_timepoint = regressor.predict(X_test_per_timepoint) 
        # then average predicted values within an epoch
        epoch_y_pred = np.mean(y_pred_per_timepoint.reshape(-1, n_timepoints), axis=1)

        print('\033[1m' + "\n test data")
        print('\033[0m')
        # R squared for timepoints
        print('R-squared timepoint data:', r2_score(y_test_per_timepoint, y_pred_per_timepoint))
    #     # sanity check - manual computation gives same R-squared
    #     actual_minus_predicted = sum((y_test_per_timepoint - y_pred_per_timepoint_1)**2)
    #     actual_minus_actual_mean = sum((y_test_per_timepoint - y_test_per_timepoint.mean())**2)
    #     r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
    #     print('R-squared timepoint data:', r2)

        # R squared for trials
        print('R-squared trial data:', r2_score(y_test_trials, epoch_y_pred))
    #     # sanity check - manual computation gives same R-squared
    #     actual_minus_predicted = sum((y_test_trials - epoch_y_pred_1)**2)
    #     actual_minus_actual_mean = sum((y_test_trials - y_test_trials.mean())**2)
    #     r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
    #     print('R-squared trial data:', r2)
        # plot y against predicted y
        sns.regplot(x=y_test_trials, y=epoch_y_pred, scatter_kws={'alpha':0.5})
        plt.xlabel('Reported confidence')
        plt.ylabel('Classifier prediction')
        plt.title('Reported trial confidence against predicted trial confidence test data')
        plt.show();


        # R squared for training data
        print('\033[1m' + "\n train data")
        print('\033[0m')
        # R squared for timepoints
        y_pred_per_timepoint_training = regressor.predict(X_train_per_timepoint) 
        print('R-squared training timepoint data:', r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
        # R squared for trials
        epoch_y_pred_train = np.mean(y_pred_per_timepoint_training.reshape(-1, n_timepoints), axis=1)
        print('R-squared training trial data:', r2_score(y_train_trials, epoch_y_pred_train))
        # plot y against predicted y
        sns.regplot(x=y_train_trials, y=epoch_y_pred_train, scatter_kws={'alpha':0.5})
        plt.xlabel('Reported confidence')
        plt.ylabel('Classifier prediction')
        plt.title('Reported trial confidence against predicted trial confidence train data')
        plt.show();

        all_scores_test.append(r2_score(y_test_per_timepoint, y_pred_per_timepoint))
        all_scores_test_trialwise.append(r2_score(y_test_trials, epoch_y_pred))
        all_scores_train.append(r2_score(y_train_per_timepoint, y_pred_per_timepoint_training))
        all_scores_train_trialwise.append(r2_score(y_train_trials, epoch_y_pred_train))


        ####################### PLOTTING ####################### 

        # plot topoplot with beta weights
        mne.viz.plot_topomap(data=betas, pos=epochs.info)
        print("Beta weights for each channel \n\n")

        # plot topoplot with sensor projections (see Parra et al paper for equation)
        y = y_pred_per_timepoint
        x = X_test_per_timepoint
        a =  y.T@x * ( 1 / (y.T@y) )
        mne.viz.plot_topomap(data=a, pos=epochs.info)
        print("Sensor projections \n\n")

    #     # shuffle ys --> gives same topoplot because my predictions are basically random and then this just plots the average voltage which happens to look like P3
    #     y = y_pred_per_timepoint
    #     np.random.shuffle(y)
    #     x = X_test_per_timepoint
    #     a =  y.T@x * ( 1 / (y.T@y) )
    #     mne.viz.plot_topomap(data=a, pos=epochs.info)
    #     print("Sensor projections shuffled ys \n\n")

    #     # shuffle xs --> gives random topoplot
    #     y = y_pred_per_timepoint
    #     x = X_test_per_timepoint
    #     def shuffle_along_axis(a, axis):
    #         idx = np.random.rand(*a.shape).argsort(axis=axis)
    #         return np.take_along_axis(a,idx,axis=axis)
    #     x = shuffle_along_axis(x, 1)
    #     a =  y.T@x * ( 1 / (y.T@y) )
    #     mne.viz.plot_topomap(data=a, pos=epochs.info)
    #     print("Sensor projections shuffled xs \n\n")

        # plot confidence signal ERP over time (use betas rather than a weights)
        across_epochs_erp = epochs.average().get_data() # gives matrix with channels x timepoints (averaging across epochs)
        weighted_erp = across_epochs_erp * betas.reshape((betas.size, 1)) # weigh the voltages by the channel weights
        average_weighted_erp = weighted_erp.mean(axis=0) # compute the average across electrodes 
        x_axis = np.arange(epoch_start, epoch_end, (epoch_end-epoch_start)/(n_timepoints))
        plt.plot(x_axis, average_weighted_erp)
        plt.show()
        print("Average voltages (across epochs) weighted by regression betas for each electrode \n\n")    





R-squared timepoint wise

In [ ]:

print('\n Average R-squared test timepoint-wise: %f' % (sum(all_scores_test) / len(all_scores_test),))



In [ ]:

print('\n Average R-squared train timepoint-wise: %f' % (sum(all_scores_train) / len(all_scores_train),))



R-squared trial wise

In [ ]:

print('\n Average R-squared test trialwise: %f' % (sum(all_scores_test_trialwise) / len(all_scores_test_trialwise),))



In [ ]:

print('\n Average R-squared train trialwise: %f' % (sum(all_scores_train_trialwise) / len(all_scores_train_trialwise),))



## Logistic regression

In [ ]:
%%capture

all_lg_scores = []

for session in sessions:
#     # accuracy >=62%
#     if session == 1:
#         participant_numbers = [1, 19, 11, 3, 17, 20, 18, 6, 2]
#     if session == 2:
#         participant_numbers = [2, 18, 7, 19, 1, 11, 17, 9]
        
    # accuracy >=64%
    if session == 1:
        participant_numbers = [1, 19, 11, 3, 20]
    if session == 2:
        participant_numbers = [2, 18, 7, 19, 1, 11, 17]

    for subject in participant_numbers:   
    #     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    #     print('\033[0m')

        # merge stimulus epochs from both sessions for the participant - TREATING EVERYONE INDIVIDUALLY
        participant_files = []
        participant_files.append('%i_%i' % (subject, session))
        epochs = load_subj_eeg(path='%s/' % input_dir, file=participant_files[0])
        
        
        # crop epochs
        epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)

        # independent variables
        # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
        X = epochs.get_data()

        n_epochs = X.shape[0]
        n_channels = X.shape[1]
        n_timepoints = X.shape[2]

        # collapse across trials and timepoints
        X = X.swapaxes(1, 2) # (n_epochs, n_times, n_channels)
        X = X.reshape(-1, n_channels) # (n_epochs x n_times, n_channels) --> n_samples, n_features

        # dependent variable as numpy array (n_epochs,)
        y = epochs.metadata['participant_confidence'].to_numpy()
        # make it binary
        med_y = np.median(y)
        y[y<=med_y] = -1
        y[y>med_y] = 1

        # extend from (n_epochs) to (n_epochs x n_timepoints, ) where confidence value stays the same
        y = np.repeat(y, n_timepoints)

        # fit regression
        reg = LogisticRegression().fit(X, y)
        score = reg.score(X, y)
    #     print(score)

    #     # regression betas
    #     betas = reg.coef_
    #     #print(betas)

    #     mne.viz.plot_topomap(data=betas, pos=epochs.info)

        all_lg_scores.append(score)


In [ ]:

print('Average accuracy logistic regression: %f' % (sum(all_lg_scores) / len(all_lg_scores),))


## Correlations between confidence and indiviudal electrodes

plotting confidence median split

In [ ]:
participant_corrs = []

for session in sessions:
#     # accuracy >=62%
#     if session == 1:
#         participant_numbers = [1, 19, 11, 3, 17, 20, 18, 6, 2]
#     if session == 2:
#         participant_numbers = [2, 18, 7, 19, 1, 11, 17, 9]
        
    # accuracy >=64%
    if session == 1:
        participant_numbers = [1, 19, 11, 3, 20]
    if session == 2:
        participant_numbers = [2, 18, 7, 19, 1, 11, 17]

    for subject in participant_numbers:   
    #     print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % subject)
    #     print('\033[0m')

        # merge stimulus epochs from both sessions for the participant - TREATING EVERYONE INDIVIDUALLY
        participant_files = []
        participant_files.append('%i_%i' % (subject, session))
        epochs = load_subj_eeg(path='%s/' % input_dir, file=participant_files[0])
    
        # crop epochs
        epochs = epochs.crop(tmin=epoch_start, tmax=epoch_end, include_tmax=True)

        # independent variables
        # get epoched data as a numpy array of shape (n_epochs, n_channels, n_times)
        # crop epochs for correlation computation
        X = epochs.get_data()

        # get epoch average for each channel (average across timepoints) (n_epochs, n_channels)
        X = np.mean(X, 2)
        # divide into even and odd trials
        X_even = X[0::2, :]
        X_odd = X[1::2, :]

        # dependent variable as numpy array (n_epochs,)
        y = epochs.metadata['participant_confidence'].to_numpy()
        # divide into odd an even trials
        y_even = y[0::2]
        y_odd = y[1::2]


        # compute correlation of every channel with confidence
        all_corrs = []
        for channel in range (X.shape[1]):
            x = X[:, channel]
            cor = np.corrcoef(y, x)[0, 1]
            all_corrs.append(cor)
        max_index = np.argmax(all_corrs)


        # plot topoploy of correlations
        print("Correlation of each channel with confidence \n\n")
        mne.viz.plot_topomap(data=all_corrs , pos=epochs.info, names=epochs.ch_names, show_names=True)


        print("\n Maximum correlation with confidence is at %s with %.3f" %(epochs.ch_names[max_index], max(all_corrs)))
    #     sns.regplot(x=y, y=X[:, max_index], scatter_kws={'alpha':0.5})
    #     plt.xlabel('Reported confidence')
    #     plt.ylabel('Average voltage')
    #     plt.show();

        participant_corrs.append(all_corrs)


        # plot high vs low confidence at the channel that max correlates with confidence
        print("\n high vs low confidence at the channel that max correlates with confidence")
        median_confidence = epochs.metadata['participant_confidence'].median()
        low_conf_evoked = epochs['participant_confidence <= %i' % median_confidence].average()
        high_conf_evoked = epochs['participant_confidence > %i' % median_confidence].average()
        evokeds = dict(high_confidence=high_conf_evoked, low_confidence=low_conf_evoked)
        mne.viz.plot_compare_evokeds(evokeds, picks=[epochs.ch_names[max_index]], invert_y=False)

    #     # plot confidence quartiles at the channel that max correlates with confidence
    #     quartiles = np.percentile(y, [25,50,75]) 
    #     y_quartiles = 4 - (quartiles[:,None] >=  y).sum(0)
    #     epochs.metadata['confidence_quartile'] = y_quartiles
    #     Q1_epochs = epochs['confidence_quartile == %i' % 1].average()
    #     Q2_epochs = epochs['confidence_quartile == %i' % 2].average()
    #     Q3_epochs = epochs['confidence_quartile == %i' % 3].average()
    #     Q4_epochs = epochs['confidence_quartile == %i' % 4].average()
    #     evokeds = dict(highest_confidence=Q4_epochs, high_confidence=Q3_epochs, low_confidence=Q2_epochs, lowest_confidence=Q1_epochs)
    #     mne.viz.plot_compare_evokeds(evokeds, picks=[epochs.ch_names[max_index]], invert_y=False)

    #     # downsampled plot
    #     epochs = epochs.decimate(20)
    #     Q1_epochs = epochs['confidence_quartile == %i' % 1].average()
    #     Q2_epochs = epochs['confidence_quartile == %i' % 2].average()
    #     Q3_epochs = epochs['confidence_quartile == %i' % 3].average()
    #     Q4_epochs = epochs['confidence_quartile == %i' % 4].average()
    #     evokeds = dict(highest_confidence=Q4_epochs, high_confidence=Q3_epochs, low_confidence=Q2_epochs, lowest_confidence=Q1_epochs)
    #     mne.viz.plot_compare_evokeds(evokeds, picks=[epochs.ch_names[max_index]], invert_y=False)



    #     # plot high vs low confidence z-scored by partner at the channel that max correlates with confidence
    #     zscore = lambda x: (x - x.mean()) / x.std()
    #     epochs.metadata['confidence_z_by_partner'] = epochs.metadata['participant_confidence'].groupby(epochs.metadata['partner']).transform(zscore)
    #     median_confidence = epochs.metadata['confidence_z_by_partner'].median()

    #     low_conf_epochs = epochs['confidence_z_by_partner <= %i' % median_confidence]
    #     high_conf_epochs = epochs['confidence_z_by_partner > %i' % median_confidence]
    #     low_conf_evoked = low_conf_epochs.average()
    #     high_conf_evoked = high_conf_epochs.average()
    #     evokeds = dict(high_confidence=high_conf_evoked, low_confidence=low_conf_evoked)
    #     mne.viz.plot_compare_evokeds(evokeds, picks=[epochs.ch_names[max_index]], invert_y=False)


    #     # compute correlation of every channel with confidence in even trials
    #     all_corrs_even = []
    #     for channel in range (X_even.shape[1]):
    #         x_even = X_even[:, channel]
    #         cor_even = np.corrcoef(y_even, x_even)[0, 1]
    #         all_corrs_even.append(cor_even)
    #     max_index_even = np.argmax(all_corrs_even)
    #     print("\n\n For even trials, maximum correlation with confidence is at %s with %.3f" %(epochs.ch_names[max_index_even], max(all_corrs_even)))
    #     cor_odd = np.correlate(y_odd, X_odd[:, max_index_even])
    #     print("\n For odd trials, correlation with confidence at %s is %.3f" %(epochs.ch_names[max_index_even], cor_odd))


    #     # compute correlation of every channel with confidence in odd trials
    #     all_corrs_odd = []
    #     for channel in range (X_odd.shape[1]):
    #         x_odd = X_odd[:, channel]
    #         cor_odd = np.corrcoef(y_odd, x_odd)[0, 1]
    #         all_corrs_odd.append(cor_odd)
    #     max_index_odd = np.argmax(all_corrs_odd)
    #     print("\n\n For odd trials, maximum correlation with confidence is at %s with %.3f" %(epochs.ch_names[max_index_odd], max(all_corrs_odd)))
    #     cor_even = np.correlate(y_even, X_even[:, max_index_odd])
    #     print("\n For even trials, correlation with confidence at %s is %.3f" %(epochs.ch_names[max_index_odd], cor_even))


    participant_correlations = np.array(participant_corrs)

In [ ]:
average_participant_correlations = np.mean(participant_correlations, 0)
average_participant_correlations

In [ ]:
np.max(average_participant_correlations)

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
print("\n Average correlation of each channel with confidence across participants")
mne.viz.plot_topomap(data=average_participant_correlations , pos=epochs.info, names=epochs.ch_names, show_names=True)
import matplotlib.pyplot as plt

In [ ]:
largest = epochs.ch_names[average_participant_correlations.argsort()[-1]]
largest

In [ ]:
second_largest = epochs.ch_names[average_participant_correlations.argsort()[-2]]
second_largest

In [ ]:
third_largest = epochs.ch_names[average_participant_correlations.argsort()[-3]]
third_largest

In [ ]:
fourth_largest = epochs.ch_names[average_participant_correlations.argsort()[-4]]
fourth_largest

In [ ]:
fifth_largest = epochs.ch_names[average_participant_correlations.argsort()[-5]]
fifth_largest

## Aggregate

In [ ]:
%%capture

participant_files = []


for session in sessions:
#     # accuracy >=62%
#     if session == 1:
#         participant_numbers = [1, 19, 11, 3, 17, 20, 18, 6, 2]
#     if session == 2:
#         participant_numbers = [2, 18, 7, 19, 1, 11, 17, 9]
        
    # accuracy >=64%
    if session == 1:
        participant_numbers = [1, 19, 11, 3, 20]
    if session == 2:
        participant_numbers = [2, 18, 7, 19, 1, 11, 17]
        

    for subject in participant_numbers:   
        participant_files.append('%i_%i' % (subject, session))

        
remerge = True
if remerge:
    epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    epochs.save('mergedData/stimulus_epoch_mastoids-epo.fif', overwrite=True)
else:
    epochs = mne.read_epochs('mergedData/stimulus_epoch_mastoids-epo.fif')

data = epochs.metadata

In [ ]:
len(data)

In [ ]:
zscore = lambda x: (x - x.mean()) / x.std()
# zscore by participant
#epochs.metadata['confidence_z_by_participant'] = epochs.metadata['participant_confidence'].groupby(epochs.metadata['participant']).transform(zscore)


In [ ]:
strategic_epochs = epochs['condition == "s"']
strategic_epochs.metadata['confidence_z_by_participant'] = strategic_epochs.metadata['participant_confidence'].groupby(strategic_epochs.metadata['participant']).transform(zscore)

nonstrategic_epochs = epochs['condition == "ns"']
nonstrategic_epochs.metadata['confidence_z_by_participant'] = nonstrategic_epochs.metadata['participant_confidence'].groupby(nonstrategic_epochs.metadata['participant']).transform(zscore)


In [ ]:
underconf_strategic_epochs = strategic_epochs['partner == "underconfident"']
underconf_strategic_epochs.metadata['confidence_z_by_participant_by_partner'] = underconf_strategic_epochs.metadata['participant_confidence'].groupby(underconf_strategic_epochs.metadata['participant']).transform(zscore)

overconf_strategic_epochs = strategic_epochs['partner == "overconfident"']
overconf_strategic_epochs.metadata['confidence_z_by_participant_by_partner'] = overconf_strategic_epochs.metadata['participant_confidence'].groupby(overconf_strategic_epochs.metadata['participant']).transform(zscore)



underconf_nonstrategic_epochs = nonstrategic_epochs['partner == "underconfident"']
underconf_nonstrategic_epochs.metadata['confidence_z_by_participant_by_partner'] = underconf_nonstrategic_epochs.metadata['participant_confidence'].groupby(underconf_nonstrategic_epochs.metadata['participant']).transform(zscore)

overconf_nonstrategic_epochs = nonstrategic_epochs['partner == "overconfident"']
overconf_nonstrategic_epochs.metadata['confidence_z_by_participant_by_partner'] = overconf_nonstrategic_epochs.metadata['participant_confidence'].groupby(overconf_nonstrategic_epochs.metadata['participant']).transform(zscore)


In [ ]:
roi = ["Pz", "CPz", "POz", "P1", "P2"]
#roi_1 = ["Pz", "CPz", "POz", "P1", "P2", "Cz"]
#roi_2 = ["P3", "CP3", "P1", "CP1", "Pz", "CPz", "P2", "CP2", "P4", "CP4"]
highest_corrs = [largest, second_largest, third_largest, fourth_largest, fifth_largest]

In [ ]:
evokeds = dict(erp=list(epochs['stimulus_left_correct', 'stimulus_right_correct'].iter_evoked()))
color_dict = {'erp':'orange'}
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=roi, colors=color_dict)


In [ ]:
partner_color_dict = {'underconfident_partner':'#4682B4', 'overconfident_partner':'#7fad1f'}

In [ ]:
## STRATEGIC
grand_average_underconf = []
grand_average_overconf = []


for session in sessions:
    if session == 1:
        participant_numbers = [1, 19, 11, 3]
    if session == 2:
        participant_numbers = [2, 18]
        
    for subject in participant_numbers:    
        grand_average_underconf.append(underconf_strategic_epochs['participant == %i' % subject].average())
        grand_average_overconf.append(overconf_strategic_epochs['participant == %i' % subject].average())


evokeds = dict(underconfident_partner=grand_average_underconf,
               overconfident_partner=grand_average_overconf)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=roi, invert_y=False, colors=partner_color_dict, 
                             ci=False, ylim=dict(eeg=[-4, 9]), vlines=[epoch_start, epoch_end])


In [ ]:
## STRATEGIC

grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_high_conf_epochs = underconf_strategic_epochs['confidence_z_by_participant >= 0']
underconf_low_conf_epochs = underconf_strategic_epochs['confidence_z_by_participant < 0']

overconf_high_conf_epochs = overconf_strategic_epochs['confidence_z_by_participant >= 0']
overconf_low_conf_epochs = overconf_strategic_epochs['confidence_z_by_participant < 0']

for session in sessions:
    if session == 1:
        participant_numbers = [1, 19, 11, 3]
    if session == 2:
        participant_numbers = [2, 18]

    for subject in participant_numbers:    
        grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
        grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
        grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
        grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())
    

evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=roi, invert_y=False, ci=False, ylim=dict(eeg=[-4, 10]))

In [ ]:
## NON-STRATEGIC
grand_average_underconf = []
grand_average_overconf = []

for session in sessions:
    if session == 1:
        participant_numbers = [20]
    if session == 2:
        participant_numbers = [7, 19, 1, 11, 17]
        
    for subject in participant_numbers:    
        grand_average_underconf.append(underconf_nonstrategic_epochs['participant == %i' % subject].average())
        grand_average_overconf.append(overconf_nonstrategic_epochs['participant == %i' % subject].average())

evokeds = dict(underconfident_partner=grand_average_underconf,
               overconfident_partner=grand_average_overconf)
mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=roi, invert_y=False, colors=partner_color_dict, 
                             ci=False, ylim=dict(eeg=[-4, 9]), vlines=[epoch_start, epoch_end])


In [ ]:
# NON-STRATEGIC 

grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_high_conf_epochs = underconf_nonstrategic_epochs['confidence_z_by_participant > 0']
underconf_low_conf_epochs = underconf_nonstrategic_epochs['confidence_z_by_participant <= 0']

overconf_high_conf_epochs = overconf_nonstrategic_epochs['confidence_z_by_participant > 0']
overconf_low_conf_epochs = overconf_nonstrategic_epochs['confidence_z_by_participant <= 0']

for session in sessions:
    if session == 1:
        participant_numbers = [20]
    if session == 2:
        participant_numbers = [7, 19, 1, 11, 17]
        
    for subject in participant_numbers:    
        grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
        grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
        grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
        grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())


evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=roi, invert_y=False, ci=False, ylim=dict(eeg=[-4, 8]))

In [ ]:
## STRATEGIC

grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_high_conf_epochs = underconf_strategic_epochs['confidence_z_by_participant >= 0']
underconf_low_conf_epochs = underconf_strategic_epochs['confidence_z_by_participant < 0']

overconf_high_conf_epochs = overconf_strategic_epochs['confidence_z_by_participant >= 0']
overconf_low_conf_epochs = overconf_strategic_epochs['confidence_z_by_participant < 0']



for session in sessions:
    if session == 1:
        participant_numbers = [1, 19, 11, 3]
    if session == 2:
        participant_numbers = [2, 18]

    for subject in participant_numbers:    
        grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
        grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
        grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
        grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())
    

# # plot with 95% confidence intervals
# evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
#                underconf_high_confidence=grand_average_underconf_high_conf,
#                overconf_low_confidence=grand_average_overconf_low_conf,
#                overconf_high_confidence=grand_average_overconf_high_conf)

# mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=roi, invert_y=False)



# plot without CIs
grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)

evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

print("\n\n strategic task, Pz")
mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)
print("\n\n strategic task, roi")
mne.viz.plot_compare_evokeds(evokeds, picks=roi, invert_y=False)
print("\n\n strategic task, highest correlating electrodes")
mne.viz.plot_compare_evokeds(evokeds, picks=highest_corrs, invert_y=False)



In [ ]:
# NON-STRATEGIC 

grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_high_conf_epochs = underconf_nonstrategic_epochs['confidence_z_by_participant > 0']
underconf_low_conf_epochs = underconf_nonstrategic_epochs['confidence_z_by_participant <= 0']

overconf_high_conf_epochs = overconf_nonstrategic_epochs['confidence_z_by_participant > 0']
overconf_low_conf_epochs = overconf_nonstrategic_epochs['confidence_z_by_participant <= 0']


for session in sessions:
    if session == 1:
        participant_numbers = [20]
    if session == 2:
        participant_numbers = [7, 19, 1, 11, 17]

    for subject in participant_numbers:      
        grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
        grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
        grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
        grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())


# # plot with 95% confidence intervals
# evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
#                underconf_high_confidence=grand_average_underconf_high_conf,
#                overconf_low_confidence=grand_average_overconf_low_conf,
#                overconf_high_confidence=grand_average_overconf_high_conf)

# mne.viz.plot_compare_evokeds(evokeds, combine='mean', picks=['Pz'], invert_y=False)



# plot without CIs
grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)

evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

print("\n\n non-strategic task, Pz")
mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)
print("\n\n non-strategic task, roi")
mne.viz.plot_compare_evokeds(evokeds, picks=roi, invert_y=False)
#print("\n\n non-strategic task, highest correlating electrodes")
#mne.viz.plot_compare_evokeds(evokeds, picks=highest_corrs, invert_y=False)

